This notebook is investigating the known methane leakages listed on the 'gas_leakage_incident.xlsx'.
The library Basemap is used to create the plots.
The plots are condenced in videos with data from surrounding days

In [11]:
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
# https://matplotlib.org/basemap/users/installing.html
from matplotlib.patches import Circle
# pip install netCDF4
import netCDF4 as nc4
from netCDF4 import Dataset
import os
import pandas as pd
from datetime import datetime
from datetime import timedelta
from datetime import date as dt
import numpy as np
import cv2
# pip install Shapely
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

print("netCDF4 version: ",nc4.__version__)

##### Settings #####
# shared folder
basepath = "data\data"
# number of days surrounding the leakage
n_days = 10
# area to plot
zoom_area = 8 # higher bigger area
# flag to keep the temp files
keep_temp = True

netCDF4 version:  1.5.3


In [2]:
# Get the list of all files on the folder
files = []
for entry in os.listdir(basepath):
    file=os.path.join(basepath, entry)
    files.append(file)

# Get the dates of the files
files_date = []
for file in files:
    netCDF4_file = Dataset(file, mode='r')
    files_date.append(datetime.fromtimestamp(netCDF4_file.time_reference_seconds_since_1970).date())

# Read Zulka's excel file
leakages_dataframe = pd.read_excel('gas_leakage_incident.xlsx')

In [6]:
# Used functions
def delta_longitude(latitude, distance):
    # function to calculate the diference in longitude given a latitude and a distance
    # approximate radius of earth in km
    R = 6373.0
    lat = np.deg2rad(latitude)
    # derivated from Haversine formula
    return np.rad2deg(2 * np.arcsin(np.sqrt( (np.sin(distance/(2*R)) ** 2) / (np.cos(lat) ** 2) )))

def find_nearest(array1, array2, value1, value2):
    # find the closest value in array, return the 2D index
    array1 = np.asarray(array1)
    array2 = np.asarray(array2)
    return np.unravel_index(np.argmin(((array1 - value1)**2 + (array2 - value2)**2)), array1.shape)

def distance(s,e):
    # Function to calculate the distance between two coordinates
    # approximate radius of earth in km
    s_lat, s_lng = s
    e_lat, e_lng = e
    R = 6373.0
    
    s_lat = s_lat*np.pi/180.0
    s_lng = np.deg2rad(s_lng)
    e_lat = np.deg2rad(e_lat)
    e_lng = np.deg2rad(e_lng)
    
    # Haversine formula
    d = np.sin((e_lat - s_lat)/2)**2 + np.cos(s_lat)*np.cos(e_lat) * np.sin((e_lng - s_lng)/2)**2
    
    return (2 * R * np.arcsin(np.sqrt(d)))

In [34]:
###### Scan ######
# check if all necessary data is present and config the list
# create a list of lists with all files from the leakages
# The high level is the leakage event
# the second level are the dates
# the third level are the files
# [[event1,[dates,[files]]]]
leakage_files = []

# scan all incidents listed on the excel file
# so far just the Assam leakage, for all remove '.head(1)'
for index, row in leakages_dataframe.iterrows():
    if pd.isna(row['latitude']): continue # avoid rows without data
    
    # get the dates of the surrounding days
    date_list = [row['date'].to_pydatetime().date() - timedelta(days=x) for x in range(n_days+1)]
    plus_date_list = [row['date'].to_pydatetime().date() + timedelta(days=x) for x in range(n_days+1)]
    date_list.reverse()
    date_list = date_list + plus_date_list[1:]
    del plus_date_list

    # check each date if there is a file of the region
    lat = row['latitude']
    long = row['longitude']
    leakage_files_second = []
    # for all dates in this location
    for date in date_list:
        sub_list = []
        file_exist = False

        # scan all files
        for i in range(len(files)):
            #### Temporal alignment ####
            # check if the date is the same
            if files_date[i] != date:
                continue
            
            # open the file safely
            file = files[i]
            try:
                netCDF4_file = Dataset(file, mode='r')
            except:
                print('Error on file: ', file)
                continue
            
            #### Spatial alignment ####
            # get the scanline with the aprox. latitude
            for scanline in range(int(netCDF4_file.groups['PRODUCT'].variables['scanline'][-1])):
                # scanline from 0 to total (e.g 3736) means latitude from -90 to 90
                # this means the data is stored from South to North (upside dowm)
                # 107 is aproximate middle of the scanline
                if netCDF4_file.groups['PRODUCT'].variables['latitude'][:][0,scanline,107] > lat: break
            # now check if the longitude is in this orbit
            minimum = netCDF4_file.groups['PRODUCT'].variables['longitude'][:][0,scanline,:].min()
            maximum = netCDF4_file.groups['PRODUCT'].variables['longitude'][:][0,scanline,:].max()
            # consider a 20 degres window to include the file
            if minimum - 10 < long and maximum + 10 > long:
                sub_list.append(file)
                file_exist = True
            netCDF4_file.close()
        # if there is no file in the folder add a note of missing file
        if not file_exist: sub_list.append('missing ' + str(row['show_text']) +' ('+ str(date)+ ')')
        
        # add the sub list to second list
        leakage_files_second.append([date,sub_list])
        del sub_list
    leakage_files.append([row['show_text'],lat,long, leakage_files_second])

# leakage_files

In [6]:
##### Create the images and the video #####

# scan the events listed to create a series of plots and make a video of it
for event in leakage_files: # first level of the list: events

    animation_list = []
    show_text,center_lat,center_long, date_list = event
    # create a temp folder
    if not os.path.exists('temp/' + show_text):
        os.makedirs('temp/' + show_text)
    
    
    for date, files2 in date_list: # second level of the list: dates
        # setup the plot
        fig = plt.figure(figsize=(16,9))
        ax = plt.gca()
        # for high res: conda install basemap-data-hires
        m = Basemap(projection='cyl',
                    urcrnrlat = center_lat + zoom_area, # top latitude
                    llcrnrlat = center_lat - zoom_area, # low latitude
                    urcrnrlon = center_long + 2*zoom_area, # right longitude
                    llcrnrlon = center_long - 2*zoom_area, # left longitude
                    resolution='i') # resolution='c' low res 'h' high res 'i' medium
        
        # Add grid lines
        m.drawparallels(np.arange(-80., 81., 10.), labels=[1,0,0,0], fontsize=10)
        m.drawmeridians(np.arange(-180., 181., 10.), labels=[0,0,0,1], fontsize=10)
        # Draw the NASA ‘Blue Marble’ image.
        m.bluemarble()
        
        # draw a circle around the event with radius of 50 km
        ax.add_patch(plt.Circle(m(center_long,center_lat), delta_longitude(center_lat,50), color='red',fill=False))
        
        for file in files2: # third level of the list: files
            # avoid missing files
            if 'missing' in file: continue
            netCDF4_file = Dataset(file, mode='r')
            # get the scanline with the aproximated latitude
            for scanline in range(int(netCDF4_file.groups['PRODUCT'].variables['scanline'][-1])):
                # scanline from 0 to total (e.g 3736) means latitude from -90 to 90
                # this means the data is stored from South to North (upside dowm)
                # 107 is aproximate middle of the scanline
                if netCDF4_file.groups['PRODUCT'].variables['latitude'][:][0,scanline,107] > center_lat: break
            
            # create a general plane to see the covered area
            l_min = scanline - 25 * zoom_area # 
            l_max = scanline + 25 * zoom_area # 400x215
            plane_data = np.array([[1800 for i in range(215)] for j in range(l_max-l_min)])

            # Convert the 3d array to a 2d array
            lons = netCDF4_file.groups['PRODUCT'].variables['longitude'][:][0,l_min:l_max,:]
            lats = netCDF4_file.groups['PRODUCT'].variables['latitude'][:][0,l_min:l_max,:]
            ch4 = netCDF4_file.groups['PRODUCT'].variables['methane_mixing_ratio'][0,l_min:l_max,:]

            # Draw reach of each file in format of a plane
            m.pcolor(lons, lats, plane_data, cmap='jet', alpha=0.05)

            # Draw ch4 data, set vmin and vmax to keep the same on all plots
            cs = m.pcolor(lons, lats, ch4, cmap='jet', vmin=1500, vmax=2000)
            netCDF4_file.close()
        # draw contours
        m.drawcoastlines()
        m.drawcountries()
        # show the colorbar
        cbar = m.colorbar(cs, location='bottom', pad="10%")
        plt.title(show_text + ' (' + str(date) + ')')
        plt.tight_layout()
        # save the image in a temp folder
        plt.savefig('temp/' + show_text + '/' + str(date)+'.png')
        plt.close(fig) # dont show the image
    
    # read all files from the temp folder and create a video
    img_array = []
    for file in os.listdir('temp/' + show_text + '/'):
        img = cv2.imread('temp/' + show_text + '/' + file)
        img_array.append(img)
        height, width, layers = img.shape
        size = (width,height)
        # delete temp files
        if not keep_temp: os.remove( 'temp/' + show_text + '/' + file)

    # create a video with 0.5 FPS
    out = cv2.VideoWriter(show_text + '.avi',cv2.VideoWriter_fourcc(*'DIVX'), 0.5, size)
    for img in img_array:
        out.write(img)
    out.release()
    del img_array
    
    # delete the temp folder
    if not keep_temp: os.rmdir('temp/' + show_text + '/')


C:\Users\natan\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: MatplotlibDeprecationWarning: 
The dedent function was deprecated in Matplotlib 3.1 and will be removed in 3.3. Use inspect.cleandoc instead.
C:\Users\natan\Anaconda3\lib\site-packages\ipykernel_launcher.py:63: MatplotlibDeprecationWarning: 
The dedent function was deprecated in Matplotlib 3.1 and will be removed in 3.3. Use inspect.cleandoc instead.


In [31]:
##### Data Fusion ######
# This cell fuses the data from two orbits in the same day
# It compares the location of the data and average if both are valid

# Same structure from leakage_files but now for data
# The high level is the leakage event
# the second level are the dates
# the third level is data
# [[event1,center_lat, center_lon,[dates,[data]]]]
leakage_data = []

for event in leakage_files: # first level of the list: events
    show_text,center_lat,center_long, date_list = event
    leak_data = []
    for date, files2 in date_list: # second level of the list: dates
        
        if len(files2) >= 2:
            # take up to 2 files
            if len(files2) == 2: file1, file2 = files2
            if len(files2) > 2:
                file1 = files2[0]
                file2 = files2[1]
            # check if the file is present
            if 'missing' in file1: continue
            if 'missing' in file2: continue
            # open the files safely
            try:
                netCDF4_file1 = Dataset(file1, mode='r')
                netCDF4_file2 = Dataset(file2, mode='r')
            except:
                print('Error on files: ')
                print(files1)
                print(files2)
            # get the scanline with the aproximated latitude
            for scanline1 in range(int(netCDF4_file1.groups['PRODUCT'].variables['scanline'][-1])):
                # scanline from 0 to total (e.g 3736) means latitude from -90 to 90
                # this means the data is stored from South to North (upside dowm)
                # 107 is aproximate middle of the scanline
                if netCDF4_file1.groups['PRODUCT'].variables['latitude'][:][0,scanline1,107] > center_lat: break
            # ~4000x215
            l_min = scanline1 - 25 * zoom_area
            l_max = scanline1 + 25 * zoom_area
            # 400x215
            
            # Convert the 3d array to a 2d array
            lons1 = netCDF4_file1.groups['PRODUCT'].variables['longitude'][:][0,l_min:l_max,:]
            lats1 = netCDF4_file1.groups['PRODUCT'].variables['latitude'][:][0,l_min:l_max,:]
            ch41 = netCDF4_file1.groups['PRODUCT'].variables['methane_mixing_ratio'][0,l_min:l_max,:]
            # get the scanline with the aproximated latitude
            for scanline2 in range(int(netCDF4_file2.groups['PRODUCT'].variables['scanline'][-1])):
                # scanline from 0 to total (e.g 3736) means latitude from -90 to 90
                # this means the data is stored from South to North (upside dowm)
                # 107 is aproximate middle of the scanline
                if netCDF4_file2.groups['PRODUCT'].variables['latitude'][:][0,scanline2,107] > center_lat: break
            
            # ~4000x215
            l_min = scanline2 - 25 * zoom_area
            l_max = scanline2 + 25 * zoom_area
            # 400x215
            
            # Convert the 3d array to a 2d array
            lons2 = netCDF4_file2.groups['PRODUCT'].variables['longitude'][:][0,l_min:l_max,:]
            lats2 = netCDF4_file2.groups['PRODUCT'].variables['latitude'][:][0,l_min:l_max,:]
            ch42 = netCDF4_file2.groups['PRODUCT'].variables['methane_mixing_ratio'][0,l_min:l_max,:]
            
            # Scan for the overlaping area
            area1 = Polygon([(lats1[0,-1], lons1[0,-1]), (lats1[0,0], lons1[0,0]), (lats1[0,-1], lons1[-1,0]), (lats1[-1,-1], lons1[-1,-1])])
            area2 = Polygon([(lats2[0,-1], lons2[0,-1]), (lats2[0,0], lons2[0,0]), (lats2[0,-1], lons2[-1,0]), (lats2[-1,-1], lons2[-1,-1])])
            x = area2.intersection(area1)
            if x.area != 0.0: # if the area is zero there is no overlaping
                # scan for coordenates in the overlap
                for i in range(len(lats2)):
                    for j in range(len(lats2[i,:])):
                        # get the coordenates
                        lat2 = lats2[i,j]
                        lon2 = lons2[i,j]
                        # library shapely can check geometrically if the coordenate is conteined
                        if x.contains(Point(lat2, lon2)):
                            # the data is overlaping
                            index = find_nearest(lats1,lons1,lat2,lon2)
                            # index is related to file1
                            dist = distance((lat2,lon2),(lats1[index],lons1[index])) # distance in km
                            # Dry-air mixing ratio of methane for cloud-free observations with a spatial
                            # resolution of 7x7km2 observed at about 13:30 local solar time
                            if dist < 9.89: # diagonal of 7x7km
                                print('found a candidate')
                                # consider to be the same value
                                # the value in ch41 is not valid
                                if np.ma.getmask(ch41)[index]: break
                                # the value in ch42 is not valid
                                if np.ma.getmask(ch42)[i,j]: break
                                
                                ####### Data normalization #########
                                # normalize the values according to the quality index
                                # get the quality index
                                # A continuous quality descriptor, varying between 0 (no data) and 1 (full quality data)
                                qi1_array = netCDF4_file1.groups['PRODUCT'].variables['qa_value'][:][0,l_min:l_max,:]
                                qi2_array = netCDF4_file2.groups['PRODUCT'].variables['qa_value'][:][0,l_min:l_max,:]
                                qi1 = qi1_array[index] # index is related to file1
                                qi2 = qi1_array[i,j]   # i, j are related to file2
                                
                                # mask the value on the ch41 and insert the average on ch2
                                value1 = ch41[index]
                                value2 = ch42[i,j]
                                ch41[index] = np.ma.masked
                                ch42[i,j] = (value1*qi1 + value2*qi2) / (qi1+qi2)
                                
            lats = np.ma.vstack((lats1, lats2))
            lons = np.ma.vstack((lons1, lons2))
            ch4 = np.ma.vstack((ch41, ch42))
            netCDF4_file1.close()
            netCDF4_file2.close()
        if len(files2) == 1: # for a single file
            # check if the file is present
            if 'missing' in files2[0]: continue
            # open the file
            netCDF4_file = Dataset(files2[0], mode='r')
            # get the scanline with the aproximated latitude
            for scanline in range(int(netCDF4_file.groups['PRODUCT'].variables['scanline'][-1])):
                if netCDF4_file.groups['PRODUCT'].variables['latitude'][:][0,scanline,107] > center_lat: break
            # ~4000x215
            l_min = scanline - 25 * zoom_area
            l_max = scanline + 25 * zoom_area
            # 400x215
            lats = netCDF4_file.groups['PRODUCT'].variables['latitude'][:][0,l_min:l_max,:]
            lons = netCDF4_file.groups['PRODUCT'].variables['longitude'][:][0,l_min:l_max,:]
            ch4 = netCDF4_file.groups['PRODUCT'].variables['methane_mixing_ratio'][0,l_min:l_max,:]
            netCDF4_file.close()
        leak_data.append([date, lats, lons, ch4])
    leakage_data.append([show_text, center_lat, center_long, leak_data])

# leakage_data

In [35]:
##### Create the images and the video #####

# scan the events listed to create a series of plots and make a video of it
for event in leakage_data: # first level of the list: events
    animation_list = []
    show_text,center_lat,center_long, date_list = event
    
    # create a temp folder
    if not os.path.exists('temp2/' + show_text):
        os.makedirs('temp2/' + show_text)
    
    for date, lats, lons, ch4 in date_list: # second level of the list: dates
        # setup the plot
        fig = plt.figure(figsize=(16,9))
        ax = plt.gca()
        # for high res: conda install basemap-data-hires
        m = Basemap(projection='cyl',
                    urcrnrlat = center_lat + zoom_area, # top latitude
                    llcrnrlat = center_lat - zoom_area, # low latitude
                    urcrnrlon = center_long + 2*zoom_area, # right longitude
                    llcrnrlon = center_long - 2*zoom_area, # left longitude
                    resolution='i') # resolution='c' low res 'h' high res 'i' medium
        
        # Add grid lines
        m.drawparallels(np.arange(-80., 81., 10.), labels=[1,0,0,0], fontsize=10)
        m.drawmeridians(np.arange(-180., 181., 10.), labels=[0,0,0,1], fontsize=10)
        
        # Draw the NASA ‘Blue Marble’ image.
        m.bluemarble()
        
        # draw a circle around the event with radius of 50 km
        ax.add_patch(plt.Circle(m(center_long,center_lat), delta_longitude(center_lat,50), color='red',fill=False))
        
        # Draw ch4 data, set vmin and vmax to keep the same on all plots
        sc = plt.scatter(lons, lats, c=ch4, cmap='jet', vmin=1500, vmax=2000, alpha=0.7, s=8)
        
        # draw contours
        m.drawcoastlines()
        m.drawcountries()
        # show the colorbar
        cbar = m.colorbar(sc, location='bottom', pad="10%")
        plt.title(show_text + ' (' + str(date) + ')')
        plt.tight_layout()
        # save the image in a temp folder
        plt.savefig('temp2/' + show_text + '/' + str(date)+'.png')
        plt.close(fig) # dont show the image
        
    # read all files from the temp folder and create a video
    img_array = []
    for file in os.listdir('temp2/' + show_text + '/'):
        img = cv2.imread('temp2/' + show_text + '/' + file)
        img_array.append(img)
        height, width, layers = img.shape
        size = (width,height)
        # delete temp files
        if not keep_temp: os.remove('temp2/' + show_text + '/' + file)

    # create a video with 0.5 FPS
    out = cv2.VideoWriter(show_text + ' - fused.avi',cv2.VideoWriter_fourcc(*'DIVX'), 0.5, size)
    for img in img_array:
        out.write(img)
    out.release()
    del img_array
    
# delete the temp folder
if not keep_temp: os.rmdir('temp2/' + show_text + '/')


In [ ]:
##### END #####

In [ ]:
# leakage_files = [['Assam leakage 27-05-2020',
#   27.604277777777778,
#   95.4046388888889,
#   [[dt(2020, 5, 17),
#     ['data\\data\\S5P_OFFL_L2__CH4____20200517T050007_20200517T064137_13433_01_010302_20200518T214726.nc',
#      'data\\data\\S5P_OFFL_L2__CH4____20200517T064137_20200517T082308_13434_01_010302_20200518T233942.nc']],
#    [dt(2020, 5, 18),
#     ['data\\data\\S5P_OFFL_L2__CH4____20200518T044109_20200518T062239_13447_01_010302_20200519T212648.nc',
#      'data\\data\\S5P_OFFL_L2__CH4____20200518T062239_20200518T080409_13448_01_010302_20200519T232443.nc']],
#    [dt(2020, 5, 19),
#     ['data\\data\\S5P_OFFL_L2__CH4____20200519T060340_20200519T074510_13462_01_010302_20200520T224831.nc']],
#    [dt(2020, 5, 20),
#     ['data\\data\\S5P_OFFL_L2__CH4____20200520T054442_20200520T072612_13476_01_010302_20200521T223938.nc',
#      'data\\data\\S5P_OFFL_L2__CH4____20200520T072612_20200520T090742_13477_01_010302_20200521T235814.nc']],
#    [dt(2020, 5, 21),
#     ['data\\data\\S5P_OFFL_L2__CH4____20200521T052543_20200521T070713_13490_01_010302_20200522T220813.nc',
#      'data\\data\\S5P_OFFL_L2__CH4____20200521T070713_20200521T084843_13491_01_010302_20200522T234241.nc']],
#    [dt(2020, 5, 22),
#     ['data\\data\\S5P_OFFL_L2__CH4____20200522T050644_20200522T064814_13504_01_010302_20200523T215000.nc',
#      'data\\data\\S5P_OFFL_L2__CH4____20200522T064814_20200522T082944_13505_01_010302_20200523T231600.nc']],
#    [dt(2020, 5, 23),
#     ['data\\data\\S5P_OFFL_L2__CH4____20200523T044745_20200523T062915_13518_01_010302_20200524T212530.nc',
#      'data\\data\\S5P_OFFL_L2__CH4____20200523T062915_20200523T081045_13519_01_010302_20200524T231032.nc']],
#    [dt(2020, 5, 24),
#     ['data\\data\\S5P_OFFL_L2__CH4____20200524T042846_20200524T061016_13532_01_010302_20200525T210215.nc',
#      'data\\data\\S5P_OFFL_L2__CH4____20200524T061016_20200524T075146_13533_01_010302_20200525T225044.nc']],
#    [dt(2020, 5, 25),
#     ['data\\data\\S5P_OFFL_L2__CH4____20200525T055117_20200525T073247_13547_01_010302_20200526T223409.nc']],
#    [dt(2020, 5, 26),
#     ['data\\data\\S5P_OFFL_L2__CH4____20200526T053217_20200526T071347_13561_01_010302_20200527T222221.nc',
#      'data\\data\\S5P_OFFL_L2__CH4____20200526T071347_20200526T085517_13562_01_010302_20200527T235240.nc']],
#    [dt(2020, 5, 27),
#     ['data\\data\\S5P_OFFL_L2__CH4____20200527T051318_20200527T065448_13575_01_010302_20200528T220114.nc',
#      'data\\data\\S5P_OFFL_L2__CH4____20200527T065448_20200527T083618_13576_01_010302_20200528T232900.nc']],
#    [dt(2020, 5, 28),
#     ['data\\data\\S5P_OFFL_L2__CH4____20200528T045418_20200528T063548_13589_01_010302_20200529T212956.nc',
#      'data\\data\\S5P_OFFL_L2__CH4____20200528T063548_20200528T081718_13590_01_010302_20200529T232047.nc']],
#    [dt(2020, 5, 29),
#     ['data\\data\\S5P_OFFL_L2__CH4____20200529T043518_20200529T061648_13603_01_010302_20200530T210438.nc',
#      'data\\data\\S5P_OFFL_L2__CH4____20200529T061648_20200529T075819_13604_01_010302_20200530T224718.nc']],
#    [dt(2020, 5, 30),
#     ['data\\data\\S5P_OFFL_L2__CH4____20200530T055749_20200530T073919_13618_01_010302_20200531T224030.nc']],
#    [dt(2020, 5, 31),
#     ['data\\data\\S5P_OFFL_L2__CH4____20200531T053849_20200531T072019_13632_01_010302_20200601T222308.nc',
#      'data\\data\\S5P_OFFL_L2__CH4____20200531T072019_20200531T090149_13633_01_010302_20200602T001734.nc']],
#    [dt(2020, 6, 1),
#     ['data\\data\\S5P_OFFL_L2__CH4____20200601T051949_20200601T070119_13646_01_010302_20200602T221321.nc',
#      'data\\data\\S5P_OFFL_L2__CH4____20200601T070119_20200601T084249_13647_01_010302_20200602T233827.nc']],
#    [dt(2020, 6, 2),
#     ['data\\data\\S5P_OFFL_L2__CH4____20200602T050048_20200602T064218_13660_01_010302_20200603T215119.nc',
#      'data\\data\\S5P_OFFL_L2__CH4____20200602T064218_20200602T082348_13661_01_010302_20200603T233138.nc']],
#    [dt(2020, 6, 3),
#     ['data\\data\\S5P_OFFL_L2__CH4____20200603T044148_20200603T062318_13674_01_010302_20200604T212730.nc',
#      'data\\data\\S5P_OFFL_L2__CH4____20200603T062318_20200603T080448_13675_01_010302_20200604T230119.nc']],
#    [dt(2020, 6, 4),
#     ['data\\data\\S5P_OFFL_L2__CH4____20200604T060418_20200604T074548_13689_01_010302_20200605T225101.nc']],
#    [dt(2020, 6, 5),
#     ['data\\data\\S5P_OFFL_L2__CH4____20200605T054517_20200605T072647_13703_01_010302_20200606T222212.nc',
#      'data\\data\\S5P_OFFL_L2__CH4____20200605T072647_20200605T090817_13704_01_010302_20200607T001647.nc']],
#    [dt(2020, 6, 6),
#     ['data\\data\\S5P_OFFL_L2__CH4____20200606T052616_20200606T070746_13717_01_010302_20200607T215556.nc',
#      'data\\data\\S5P_OFFL_L2__CH4____20200606T070746_20200606T084916_13718_01_010302_20200607T234433.nc']]]]]